<h1> Imports </h1>

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os

/Users/gpoulsen@apamail.org/Library/Python/3.10/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


<h1> County Population Data </h1>

<h3> Helper Functions </h3>

In [2]:
def dropOutliers(df):

    toRemove = [] #List of outliers to remove

    yrs = df['Year'].unique() 
    g = df.groupby('Year') #Define groups for every year we'll examine

    #Loop through groups
    for y in yrs:

        s = g.get_group(y)['Pop'] #Get Group

        iq = s.describe().loc['75%'] - s.describe().loc['25%'] #Calculate the inner quartile
        ub = s.describe().loc['75%'] + 1.5 * iq #Create Lower Bound considered outliers
        lb = s.describe().loc['25%'] - 1.5 * iq #Creatre Uppper Bound considered outliers

        toRemove.extend(list(s[~s.between(lb,ub)].index)) #Store outliers to remove later
        df = df[~df.index.isin(toRemove)]
        
    return df

def cleanFrame(df):

    df['Pop'] = df['Pop'].astype(int) #Format response as int

    df.drop(columns=['State','County']) #Drop Numeric Identifiers
    df['County'] = df['Name'].str.split(",").str[0] #Split Name Column: County
    df['State'] = df['Name'].str.split(",").str[1] #Split Name Column: State

    df.index = df['State'] + "-" + df['County'] + "-" + df['Year'] #Set Unique Identifier
    df = df.drop(columns='Name') #Drop Name Column

    df = df[['County','State','Year','Pop']] #Reorder Columns

    return df

<h1> Requesting County Populations from 2005 to 2021 </h1>

In [3]:
apiKey = 'fc8e94a76785a060f4aba9a659b4c84157f46dea' #I should omit this later

cols = ['Name','Pop','State','County','Year'] #Labels for my df
df = pd.DataFrame(columns=cols) #Create empty frame

yrs = [*range(2005,2022)] #Consider all years
yrs.remove(2020) #except for 2020 which was not released due to issues with Covid

#Get All Available Years
for y in yrs:

    url = 'https://api.census.gov/data/' + str(y) + '/acs/acs1?get=NAME,B01001_001E&for=county:*&key=' + apiKey #URL for Query
    response = requests.request("GET",url) 
    
    cLen = len(response.json()[1:]) 
    year = np.array([str(y) for x in range(cLen)]).reshape(cLen,1) #Create a year column for labeling our response

    dt = np.array(response.json()[1:]) #Format response as np array
    dt = np.hstack((dt,year)) #Add a year column to data
    dt = pd.DataFrame(dt, columns=cols) #Format as a frame

    df = pd.concat([df,dt],ignore_index=True) #Concatonate w/ our loaded data as of thus far

print("Avg. num. of data pts. for each region " + str(df['Name'].value_counts().mean())) #A rough summurative check on what we've loaded

Avg. num. of data pts. for each region 15.2995337995338


In [4]:
df.to_parquet('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Integration/Extracted Data/Population.gzip')

<h1> County Migration Data </h1>

<h3> Helper Functions </h3>

In [5]:
def processNewFormat(fn): #Helper function. We feed in the New files pathway and it returns tabularized data

    df = pd.read_csv(fn,encoding='latin1') #Load in file
    res = pd.DataFrame(columns=["State","County","Year","Total Exemptions"]) #Set Expected Dataframe

    yr = fn.split('/')[-1] #Determine our year
    yr = [x for x in yr if x.isdigit()]
    yr = '20' + yr[0] + yr[1] + ' to 20' + yr[2] + yr[3]

    # Rename so it's columns are accessable
    cnt = 0
    for c in df.columns:

        df = df.rename(columns={c:cnt})
        cnt += 1

    df = df[ (df[2] == 96) & (df[3]==0)].reset_index(drop=True) #Filter down to summurative data

    for idx, row in df.iterrows(): # Loop through frame

        state = df.loc[idx,0] # Get State Code
        county = df.loc[idx,1] # Get County Code
        exemptTotal = df.loc[idx,7] #Get Exempt Totals

        if('outflow' in fn): #Set negative if we are dealing with outflow

            exemptTotal = -exemptTotal

        res.loc[len(res)] = [state,county,yr,exemptTotal] #Append

    return res

def processOldFormat(fn): #Helper function. We feed in the old files pathway and it returns tabularized data

    if(('0910' in fn) or ('1011' in fn)): #Format Varies by Year
        df = pd.read_excel(fn,skiprows=5) # Read in data
        # df = df[df[-1] != 'd']
    else:
        df = pd.read_excel(fn,skiprows=7,skipfooter=3) # Read in data
        # df = df[df['Unnamed: 7'] != 'd'] #Get rid of missing data

    res = pd.DataFrame(columns=["State","County","Year","Total Exemptions"]) #Set Expected Dataframe

    yr = fn.split('/')[-1] #Determine our year
    yr = [x for x in yr if x.isdigit()]
    yr = '20' + yr[0] + yr[1] + ' to 20' + yr[2] + yr[3]

    # Rename so it's columns are accessable
    cnt = 0
    for c in df.columns:

        df = df.rename(columns={c:cnt})

        if( (cnt != 4) & (cnt != 5) ): #Convert to numeric data if appropriate

            df = df[~df[cnt].astype(str).str.contains('d')] #Drop Missing Data
            df[cnt] = df[cnt].astype(int)

        cnt += 1

    df = df[ (df[2] == 96) & (df[3]==0)].reset_index(drop=True) #Filter down to summurative data

    for idx, row in df[df.index > 0].iterrows(): # Loop through frame

        state = df.loc[idx,0] # Get State Code
        county = df.loc[idx,1] # Get County Code
        exemptTotal = df.loc[idx,7] #Get Exempt Totals

        if( (((yr == '2005 to 2006') | (yr == '2006 to 2007')) & ('o' == fn.split('/')[-1][8])) | (((yr != '2005 to 2006') & (yr != '2006 to 2007')) & ('o' == fn.split('/')[-1][6])) ): #Check posiitonal indicator for inflow vs outflow file for based on formats for each year

            exemptTotal = -exemptTotal
            
        res.loc[len(res)] = [state,county,yr,exemptTotal]

    return res

<h3> Extraction Process </h3>

In [6]:
df = pd.DataFrame(columns=["State","County","Year","Total Exemptions"]) #Set Expected 
rd = '/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Extraction/Source Data/IRS Migration' # Root Directory
 
for dir in os.listdir(rd): # Loop through root directory
   
    if(dir == ".DS_Store"): # Skip if we hit .DS_Store
        continue

    cd = rd + "/" + dir #Get Child Directory

    if('migration' in dir): #Handling the new format
        
        fnIn = [s for s in os.listdir(cd) if "countyinflow" in s][0] #Get Inflow filename
        fnOut = [s for s in os.listdir(cd) if "countyoutflow" in s][0] #Get Outflow filename

        df = pd.concat([df,processNewFormat(cd + "/" + fnIn)], ignore_index=True) #Process the In
        df = pd.concat([df,processNewFormat(cd + "/" + fnOut)], ignore_index=True) #Process the Out

    else: #Handling the old format

        for fn in os.listdir(cd): #Loop through every inflow and outflow file

            if(fn.split('.')[1] == 'xls'): #If excel process

                df = pd.concat([df,processOldFormat(cd + "/" + fn)],ignore_index=True)

df['Flow'] = 'In'
df.loc[df['Total Exemptions'] < 0,'Flow']  = 'Out'
df['County'] = df['County'].astype(str).str.zfill(3)
df['State'] = df['State'].astype(str).str.zfill(2)
df['Year'] = df['Year'].str.split(" ").str[0]
df.index = df['State'].astype(str) + df['County'].astype(str) + "-" + df['Year'] + "-" + df['Flow'] #Set index

<h3> Quick Meta-Analysis on Extraction </h3>

In [7]:
print("Distribution for Count of State-County-Year Records: \n"  + str(df[['State','County','Year']].value_counts().value_counts() / df[['State','County','Year']].value_counts().value_counts().sum()))

Distribution for Count of State-County-Year Records: 
2    0.998607
1    0.001393
dtype: float64


<p> Ideally the distribution should be P(2) = 1, but P(2) approx. .999. We'll drop records that are missing an opposing flow record. </p>

In [8]:
sr = df[['State','County','Year']].value_counts() #Store series
idx = list(sr[sr<2].index) #Split out index of those we need to filter out

sts = [x[0] for x in idx]
cnts = [x[1] for x in idx]
yrs = [x[2] for x in idx]

print("Rows before dropping: " + str(len(df))) #We should lose 66 rows

for i in range(len(sts)):

    df = df[ (df['State'] != sts[i]) | (df['County'] != cnts[i]) | (df['Year'] != yrs[i])] #Filter our rows determined by idx seperated out

#I'm certain there is a cleaner way to do this; I may circle back to this if time permits.

print("Rows after dropping: " + str(len(df)))

Rows before dropping: 94696
Rows after dropping: 94630


In [9]:
df.groupby('Year')['Total Exemptions'].sum()

Year
2005    124030
2006    132741
2007    144342
2008    134215
2009    -14557
2010    -18433
2011    -60470
2012     -1007
2013    -19617
2014     17235
2015    -38692
2016    -23693
2017     51344
2018     19521
2019     18455
Name: Total Exemptions, dtype: object

<p> Domestic migration is a closed system, so net gains or losses are attributable to recorded foreign migration, which is reflected in the data set. </p>

<h3> Transforming Datapoints into net migration values </h1>

In [10]:
df = df.groupby(['State','County','Year'])['Total Exemptions'].sum().to_frame().reset_index() 
df.index = df['State'].astype(str) + df['County'].astype(str) + "-" + df['Year'] #Set index
df = df.rename(columns={'Total Exemptions':'Net Migration'})
df.to_parquet('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Integration/Extracted Data/Migration.gzip',index=0) #Save the data

<h1> HPI Data </h1>

<h3> Helper Functions </h3>

In [2]:
def cleanHPI(df):

    df = df[df['hpi_flavor'] == 'all-transactions'] #Look at all transactions
    df = df[df['level'] == 'MSA'] #Look at Metropoliton Statistical Areas

    df['city'] = df['place_name'].str.split(',').str[0]
    df['state'] = df['place_name'].str.split(',').str[1].str.strip() #Split Out

    df['city'] = df['city'].str.split('-').str[0] 
    df['state'] = df['state'].str[0:3] #Take the first city and state indicator for the region

    df = df[['city','state','place_id','yr','period','index_nsa']] #Keep relevant fields
    df = df.groupby(['city','state','place_id','yr'])['index_nsa'].mean().to_frame().reset_index() #Integrate quarterly indexes with a mean
    df.loc[df['state'].str.len() > 2,'state'] = df[df['state'].str.len() > 2]['state'].str[0:2] # Clean out hyphens
    
    return df

def cleanConvertor(df):

    df['county_fips'] = df['county_fips'].astype(str)
    df['county_fips'] = df['county_fips'].apply(lambda x: '0' + x if len(x) < 5 else x)

    df = df[['city','state_id','county_fips']]
    df = df.rename(columns={'state_id':'state'})

    return df

<h3> Load & Clean </h3>

In [22]:
df = pd.read_csv('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Extraction/Source Data/HPI/HPI_master (1).csv')
cnv = pd.read_csv('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Extraction/Source Data/HPI/uscities.csv') #Load

df = cleanHPI(df)
cnv = cleanConvertor(cnv) #Clean

df = df.merge(right=cnv,on=['city','state'],how='left') #Merge Data to get FIPS

<h3> Missing Analysis </h3>

In [23]:
ms = df[df['county_fips'].isnull()] #Get Nulls On Merge

print("Percentage of locations with no attatched county:" + str(len(ms[['city','state']].drop_duplicates()) / len(df[['city','state']].drop_duplicates())))

Percentage of locations with no attatched county:0.022277227722772276


<h3> Missing Quick Fix (I did this manually, as there were so few) </h3>

In [24]:
ms = ms[['city','state']].drop_duplicates().values #Get Nulls from Left Frame
fpCds = [25001, 16001, 17097, 21111, 42091, 36059, 33015, 15003, 37067] #Fill w/ Looked uUp Values
cnt = 0

for rw in ms:

    df.loc[(df['city'] == rw[0]) & (df['state'] == rw[1]),'county_fips'] = fpCds[cnt]
    cnt+=1

print("The # Missing: " + str(len(df[df['county_fips'].isnull()]))) 

The # Missing: 0


In [30]:
df.groupby('yr')['index_nsa'].median()

yr
1975     28.21250
1976     33.40750
1977     39.16000
1978     45.64000
1979     52.86500
1980     56.59625
1981     58.94500
1982     59.14500
1983     62.23750
1984     64.77750
1985     68.02000
1986     72.57750
1987     76.99750
1988     79.45375
1989     82.83250
1990     85.47625
1991     87.93500
1992     91.40500
1993     94.77500
1994     98.46250
1995    102.35125
1996    105.98875
1997    109.54875
1998    114.23000
1999    118.15125
2000    123.15375
2001    130.79750
2002    136.56125
2003    142.97750
2004    152.51875
2005    163.42250
2006    175.54500
2007    183.18625
2008    182.73625
2009    176.59250
2010    169.14250
2011    164.02375
2012    163.80625
2013    165.71500
2014    171.97750
2015    178.06500
2016    185.74000
2017    194.20250
2018    204.77500
2019    213.82750
2020    224.91875
2021    252.70750
2022    292.30000
Name: index_nsa, dtype: float64

In [15]:
df['county_fips'] = df['county_fips'].astype(str) #Cast to str so that it can be saved in parquette
df.index = df['county_fips'] + "-" + df['yr'].astype(str)
df.to_parquet('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Urban-Growth/Project/Integration/Extracted Data/Housing Index.gzip',index=0) #Save the data